In [16]:
from json import load,dump

In [17]:
def build_inbound(inbound:list)->str:
    return ( "([" + ", ".join(inbound) + "])" if len(inbound) > 1 else "(" + inbound[0] + ")" ) if len(inbound) else ""

def set_argument(argument,config):
    value = config['value']
    try:
        value = eval(value)
    except:
        pass
    value = ( None if value == 'None' else f"'{value}'" ) if config['type'] == 'text' else value
    return f"    {argument}={value.__repr__()},"
    
def build_arguments(arguments:dict)->str:
    arguments = '\n'.join([ set_argument(arg,cnf) for arg,cnf in arguments.items() ])
    return arguments

def build_default(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound']) if layer['type']['name'] != 'Input' else '' 
    
    return f"""{layer['id']} = {layer['type']['_class']}.{layer['type']['name']}(
{arguments}
){inbound} #end-{layer['id']}
"""

def build_application(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound']) if layer['type']['name'] != 'Input' else '' 
    return f"""{layer['id']} = applications.{layer['type']['_class']}(
    input_tensor={layer['connections']['inbound'][0]},
{arguments}
    include_top=False
).output #end-{layer['id']}
"""

def build_model(layer,build_config,*args,**kwargs)->str:
    build_config['train_config']['model'] = layer
    return f"""{layer['id']} = keras.Model(
    [ {', '.join(build_config['input_nodes'])}, ],
    [ {', '.join(layer['connections']['inbound'])}, ]
) #end-{layer['id']}
"""

def build_compile(layer,build_config,*args,**kwargs)->str:
    build_config['train_config']['compile'] = layer
    model,*_ = [node for node in layer['connections']['inbound'] if "model" in node]
    metrics = layer['arguments']['metrics']['value']
    metrics = "[\"" + '", "'.join(metrics) + "\"]" if len(metrics) else 'None'
    
    train_config = build_config['train_config']
    return f"""{train_config['optimizer']['value'] if train_config['optimizer'] else ''}
{model}.compile(
    optimizer={train_config['optimizer']['id'] if train_config['optimizer'] else "'"+layer['arguments']['optmizer']['value']+"'"},
    loss='{layer['arguments']['loss']['value']}',
    metrics={metrics}
) #end-{layer['id']}
"""

def build_train(layer,build_config,*args,**kwargs)->str:
    callbacks = [callback['value'] for callback in build_config['train_config']['callbacks']]
    callback_ids = [callback['id'] for callback in build_config['train_config']['callbacks']]
    build_config['train_config']['train'] = layer
        
    print (build_config['train_config']['model'])
        
    return f"""{"".join(callbacks)}
{build_config['train_config']['model']['id']}.fit(
    x={build_config['train_config']['dataset']['id']}.train_x,
    y={build_config['train_config']['dataset']['id']}.train_y,
    batch_size={layer['arguments']['batch_size']['value']},
    epochs={layer['arguments']['epochs']['value']},
    validation_data=( {build_config['train_config']['dataset']['id']}.test_x, {build_config['train_config']['dataset']['id']}.test_y ),
    callbacks=[ tfgui, {', '.join(callback_ids)} ]
) #end-{layer['id']}
"""

build_functions = {
    "default":build_default,
    "Model":build_model,
    "Compile":build_compile,
    "Train":build_train,
    "Application":build_application
}

In [18]:
with open("../temp/graph.json","r") as build_file:
    build_config = load(build_file)
    
inputs = []
train_config = {
    "dataset":None,
    "optimizer":None,
    "loss":None,
    "callbacks":[],
    
    "model":None,
    "compile":None,
    "train":None
}


for _id,config in build_config.items():
#     print (config['type']['name'])
    if config['type']['name'] == 'Input':
        inputs.append(_id)
        
    elif config['type']['name'] == 'Dataset':
        train_config['dataset'] = _id
             
    elif config['type']['name'] == 'Model':
        train_config['model'] = _id
        
    elif config['type']['name'] == 'Loss':
        train_config['loss'] = _id
    
    if config['type']['_class'] == 'optimizers':
        train_config['optimizer'] = _id
        
    elif config['type']['_class'] == 'callbacks':
        train_config['callbacks'].append(_id)
   

build_config['train_config'] = train_config
build_config['input_nodes'] = inputs
levels = [ set() for i in range(len(build_config))]
def setLevel(node,config,di=0):
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            setLevel(next_node,build_config,di+1)

for inp in inputs:
    setLevel(inp,build_config,0)
    
build_config['levels'] = levels
levels = [list(level) for level in levels if len(level)]
for key,val in train_config.items():
    if val != None:
        if key == 'dataset':
            train_config['dataset'] = {
                "id":val,
                "value":build_config[val]['arguments']['dataset']['value']
            }
        elif key == 'optimizer':
            train_config['optimizer'] = {
                'id':val,
                'value':build_default(build_config[val],build_config)+'\n'
            }
        elif key == 'callbacks':
            callbacks = []
            for callback in train_config['callbacks']:
                callbacks.append({
                    "id":callback,
                    "value":build_default(build_config[callback],build_config)
                })
            train_config['callbacks'] = callbacks
            
build = ''

for level in levels:
    for layer in level:
        layer = build_config[layer]
        if layer['type']['name'] in build_functions: 
            build += build_functions[layer['type']['name']](layer,build_config) + '\n\n'
        else:
            build += build_functions['default'](layer,build_config) + '\n\n'
            
build = build[:-2]
code = """#-*- Code generated by Tensorflow GUI -*-
#import
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics,callbacks,applications

#end-import

{dataset}
{build}
""".format(
    dataset=train_config['dataset']['value'],
    build=build
)

{'id': 'model_1', 'name': 'Model 1', 'type': {'name': 'Model', '_class': 'models'}, 'pos': {'x': 197, 'y': 554, 'offsetX': 42, 'offsetY': 20}, 'connections': {'inbound': ['dense_1'], 'outbound': ['compile_1']}, 'width': 84, 'arguments': {}}


In [19]:
with open("../data/example_train.json","w+") as file:
    file.write(build_config.__str__())

In [20]:
print (code)

#-*- Code generated by Tensorflow GUI -*-
#import
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics,callbacks,applications

#end-import

"""
Note : Don't change dataset id.

All the required packages have been imported with their standard namespaces.

tensorflow as tf
keras as keras
pandas as pd
numpy as np

from sklearn.model_selection , train_test_split
"""


#dataset id=dataset_1
class Dataset:
    """
    Dataset will be used in training 

    The dataset object needs to have following attributes

    train_x : np.ndarray -> Training features
    train_y : np.ndarray -> Training labels 
    test_x : np.ndarray -> Testing features
    test_y : np.ndarray -> Testing labels

    validate : bool -> Weather use validation data or not

    batch_size : int -> Batch size
    epochs : int -> Number of epochs
    batches : int -> Number of batches ( Will be calculated automatically )
   

In [15]:
with open("../data/example_code.py","w+") as file:
    file.write(code)

In [1]:
import tensorflow as tf

In [2]:
tf.keras.applications.VGG16().summary()

 15138816/553467096 [..............................] - ETA: 11:53

KeyboardInterrupt: 